In [ ]:
import pandas as pd
import matplotlib as mpl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor as RFR

In [ ]:
fucci = pd.read_csv("./path/to/FucciInt_LiveCell.csv",index_col=0)
fucci.columns=["Green","Red","Time","Cell"]
cluster_1=['Cell9',
  'Cell8',
  'Cell14',
  'Cell11',
  'Cell7',
  'Cell3',
  'Cell1',
  'Cell10']
cluster_2=['Cell4',
  'Cell2',
  'Cell13',
  'Cell5',
  'Cell15',
  'Cell6',
  'Cell12']
rf_data_grn = []
gene_indiex = HighEdges
for cell_key in range(1,16):
    ckey = "Cell"+str(cell_key)
    rf_data_grn.append([])
    for key in gene_indiex:
        reg, resp = key.split(">")
        if reg in GRNs["Cell{}".format(cell_key)].index and resp in GRNs["Cell{}".format(cell_key)].columns:
            rf_data_grn[cell_key-1].append(GRNs["Cell{}".format(cell_key)].loc[reg,resp])
        else:
            rf_data_grn[cell_key-1].append(0)
rf_data_grn = pd.DataFrame(rf_data_grn,index = ["Cell"+str(cell_key) for cell_key in range(1,16)],columns = gene_indiex)

rf_data_fucci = []
for cell_key in range(1,16):
    rf_data_fucci.append([])
    ckey = "Cell"+str(cell_key)
    cell_df_fucci = fucci[fucci["Cell"]==ckey]
    for rg in ["Green","Red"]:
        for t in range(4):
            rf_data_fucci[cell_key-1].append(cell_df_fucci[cell_df_fucci["Time"]==(t)*2.5][rg].values[0])
rf_data_fucci = pd.DataFrame(rf_data_fucci,index = ["Cell"+str(cell_key) for cell_key in range(1,16)])

rg = RFR(n_jobs=-1, random_state=16,n_estimators=200) 
rg.fit(rf_data_grn, rf_data_fucci)

pertubated_rf_data_grn =[]
for cell_key in range(1,16):
    ckey = "Cell"+str(cell_key)
    pertubated_rf_data_grn.append([])
    for key in gene_indiex:
        reg, resp = key.split(">")
        if (reg == "SMAD2")&(resp=="CDKN1A"):
            pertubated_rf_data_grn[cell_key-1].append(0)
        elif (reg == "EP300")&(resp=="CDKN1A"):
            pertubated_rf_data_grn[cell_key-1].append(0)
        elif reg in GRNs["Cell{}".format(cell_key)].index and resp in GRNs["Cell{}".format(cell_key)].columns:
            pertubated_rf_data_grn[cell_key-1].append(GRNs["Cell{}".format(cell_key)].loc[reg,resp])
        else:
            pertubated_rf_data_grn[cell_key-1].append(0)
pertubated_rf_data_grn = pd.DataFrame(pertubated_rf_data_grn,index = ["Cell"+str(cell_key) for cell_key in range(1,16)],columns = gene_indiex)
pertubated_fucci_pred = pd.DataFrame(rg.predict(pertubated_rf_data_grn),index=rf_data_fucci.index)


In [ ]:
for i,j in zip(rf_data_fucci.values, pertubated_fucci_pred.values):
    original_trj = pd.DataFrame([i[:4],i[4:]],index=["Green","Red"]).T
    pertubated_trj = pd.DataFrame([j[:4],j[4:]],index=["Green","Red"]).T
    fig,ax = plt.subplots(figsize=(2,2))
    original_trj.plot.scatter(x=0,y=1,ax=ax)
    pertubated_trj.plot.scatter(x=0,y=1,ax=ax,c="k")
    plt.show()